╔══<i><b>&nbsp;Alai-DeepLearning&nbsp;</b></i>══════════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 9_cnn_basis**
# Homework 3. Cifar10 을 CNN 으로 학습시키기


### _Objective_
1. homework 목적
Color Image 인 Cifar10 데이터를 CNN으로 학습시켜 봅니다.
모델을 작성 후 모델을 학습시켜 80% 이상 정확도를 가진 분류기를 만들어 봅니다. 


╚═══════════════════════════════════════════════╝

---------

# 실행 순서 
------
+ ### 1. DataDownload 
+ ### 2. EDA 
 #### 1. train, test pixel 값의 EDA
 #### 2. train ,test class 별 데이터 갯수 

+ ### 3. CNN 구성
 - **layer 별 학습**
 - **output ch 갯수 별 학습**
    - [16, 32, 64, 128 ,26]
 - **layer 순서별 kernel size** 
    - (5,5) (3,3), (3,3), (3,3), (3,3)
 - **Regularization 적용**
    - L2, Regularization : 0.0005
 - **batch 별 학습**
    - 120 
 - **Optimizer**
    - Gradient Descent optimizer 

+ ### 4. metric 측정 
 - (tensorboard accuracy, loss step별로 측정)

+ ### 5. analysis 
 - 각 층별 weights, bias 평균의 변화 측정 (step 별)*이탤릭체 텍스트*
------


# 실행방법: 

- DNN 파트를 CNN으로 변경한 후 다시 최적의 파라미터를 찾습니다.<br>
그 후 가장 최적의 hyper parameter 을 찾은 후 optimizer 을 바꿔가며 최적화 해 보세요!


### 1. DataDownLOAD 

In [0]:
import keras 
cifar10 = keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
import matplotlib.pyplot as plt 
import numpy as np 

In [0]:
# 이미지 데이터 셋 sample 
plt.imshow(train_images[0])
plt.show()
# 라벨 데이터 셋 sample 
print('label sample :', train_labels[0])

#### Image EDA 

1. train 최대값 최소값 분포도 / 평균, 분산 

In [0]:
# Train Image 
n_train = len(train_images)
seq_length = np.prod(train_images.shape[1:])
h, w, ch = train_images.shape[1:]

# 모든 이미지의 min, max 값 확인
train_min = train_images.reshape([-1,seq_length]).min(axis=1)
train_max = train_images.reshape([-1,seq_length]).max(axis=1)

train_mean = train_images.reshape([-1,seq_length]).mean(axis=1)
train_std = train_images.reshape([-1,seq_length]).std(axis=1)

print('# train : {}'.format(n_train))
print('seq_length : {}'.format(seq_length))
print('h, w, ch : {} {} {}'.format(h, w, ch))

In [0]:
# Test Image 
n_test = len(test_images)
seq_length = np.prod(test_images.shape[1:])

test_min = test_images.reshape([-1,seq_length]).min(axis=1)
test_max = test_images.reshape([-1,seq_length]).max(axis=1)

test_mean = test_images.reshape([-1,seq_length]).mean(axis=1)
test_std = test_images.reshape([-1,seq_length]).std(axis=1)

print('# test : {}'.format(n_test))
print('seq_length : {}'.format(seq_length))
print('h, w, ch : {} {} {}'.format(h, w, ch))

In [0]:
plt.figure(figsize=(10,10))
plt.scatter(train_min, train_max, s=3, alpha=0.5, label='train')
plt.scatter(test_min, test_max, s=3, alpha=0.5, label='test')
plt.legend()
plt.show()

plt.hist(train_mean, alpha=0.3, label='train')
plt.hist(test_mean, alpha=0.3, label='test')
plt.title('Mean')
plt.legend()
plt.show()

plt.hist(train_std, alpha=0.3, label='train std')
plt.hist(test_std, alpha=0.3, label='test std')
plt.title('Stddev')
plt.legend()
plt.show()


#### Label EDA 

In [0]:
print('train label data shape : {}'.format(train_labels.shape))
print('test label data shape : {}'.format(test_labels.shape))
print('label 을 scalar 형태로 reshape 합니다 ex){5000,1} -> {5000}')
train_labels = train_labels.reshape(-1)
test_labels = test_labels.reshape(-1)
print('train label data shape : {}'.format(train_labels.shape))
print('test label data shape : {}'.format(test_labels.shape))

print(np.unique(train_labels, return_counts=True))
print(np.unique(test_labels, return_counts=True))

num_outputs = len(np.unique(test_labels, return_counts=True)[0])

### Normalize and Normalization checking  

In [0]:
np.unique([train_labels],return_counts=True)

# Normalization
train_images = train_images.reshape(-1,seq_length) / 255.
test_images = test_images.reshape(-1,seq_length) / 255.

# Check Normalization 
def chk_norm(input_data, min_value=0, max_value=1):
    """
    input_data : Ndarray, 
    min_value : int, input_data의 모든 element는 min_value 이상의 값을 가져야함 
    max_value : int, input_data의 모든 element는 max_value 이하의 값을 가져야함 

    description: input_data의 모든 element가 특정 범위(min_value ~ max_value)에 있는지 확인합니다.
    """
    input_data = np.asarray(input_data)
    assert np.all(input_data <= max_value) & np.all(input_data >= min_value), 'normalize가 잘못 되었습니다.'


chk_norm(train_images)    
chk_norm(test_images)

## Data reshape 

In [0]:
train_images=train_images.reshape([-1, h, w, ch])
test_images=test_images.reshape([-1, h, w, ch])

## Input layer

In [0]:
import tensorflow as tf 
num_inputs = h*w # MNIST Input size
num_outputs = 10 # The number of Label : 26
image_ch = ch

tf.reset_default_graph()

# Data를 받아오는 placeholder
x = tf.placeholder(tf.float32, shape=[None, h, w, image_ch],name='x')

#labels shape : [1,3,7,7,4,..]
labels_cls = tf.placeholder(tf.int32, shape=[None], name='labels') 

# scalar 을 onehot-vector 형태로 변환합니다.
labels = tf.one_hot(labels_cls, depth=num_outputs)

# Dropout 시 units이 0으로 될 확률.
drop_rate = tf.placeholder(tf.float32, shape=[], name='drop_rate')

# Learning Rate
learning_rate = tf.placeholder_with_default(0.1,shape=(), name='learning_rate')

# 학습 flag 
phase_train = tf.placeholder(tf.bool, shape=[], name='phase_train')

# TODO: shape 을 고정하지 않고 하는 방법도 생각해 보세요
num_layers=5
l2_lambda = tf.placeholder_with_default(0.0, shape=[], name='l2_lambda') 
l1_lambda = tf.placeholder_with_default(0.0, shape=[], name='l1_lambda')

# BUILD MODEL

In [0]:
# CNN Archithecture
# layer 순서별 kernel_size 
kernel_sizes = [(5,5),(3,3),(3,3),(3,3)]
conv_layer_units = [16, 32, 64, 128]

# layer 길이 
n_layers = len(conv_layer_units)

#augmentation 
def train_aug(img):
    img=tf.image.random_brightness(img, 0.01)
    #img=tf.image.random_contrast(img, 0, 0.1)
    img=tf.image.random_flip_left_right(img)
    img=tf.image.random_flip_up_down(img)
    img=tf.image.random_saturation(img, lower=0, upper=0.1)
    img=tf.image.random_hue(img, 0.1)
    
    return img

def true_fn():
    return tf.map_fn(train_aug, x)

def false_fn():
    return x

aug_x = tf.cond(phase_train , true_fn, false_fn)




# conv layer 구축
in_ch = image_ch
layer = aug_x



for layer_idx in range(n_layers):
    with tf.variable_scope('conv_{}'.format(layer_idx)):

        # output channels
        out_ch = conv_layer_units[layer_idx]
        
        # kernel 
        k_h, k_w = kernel_sizes[layer_idx]
        kernel = tf.Variable(tf.random.normal([k_h, k_w, in_ch, out_ch],
                                              stddev=np.sqrt(2/num_inputs)),
                             name='weight')

        # 추후 weights 만 꺼내 사용하기 위해 weights 을 따로 보관함.          
        tf.add_to_collection('weights', kernel)
        
        # bias 
        b = tf.Variable(tf.zeros([out_ch]), name='bias')

        # Convolution
        layer = tf.nn.conv2d(layer, kernel, strides=[1,1,1,1], padding='SAME') + b
        
        # Activation
        layer = tf.nn.relu(layer)
        
        # max pooling : 이미지를 1/2 로 줄입니다. 
        layer = tf.nn.max_pool(layer, ksize=[1,2,2,1], strides=[1,2,2,1],
                              padding='SAME')
            
        in_ch = conv_layer_units[layer_idx]

# Top convolution.
top_conv = tf.identity(layer, 'top_conv')
top_conv_shape = list(map(int, top_conv.get_shape()[1:]))

flat_layer = tf.reshape(top_conv, [-1, np.prod(top_conv_shape)])

In [0]:
# Fully Connected Layer Archithecture
fc_layer_units = [256, 256 ,num_outputs]
n_fc_layers = len(fc_layer_units)
layer = flat_layer
in_ch= int(flat_layer.get_shape()[-1])

for layer_idx in range(n_fc_layers):
    with tf.variable_scope('fc_{}'.format(layer_idx)):
        
        # output channels
        out_ch = fc_layer_units[layer_idx]
        
        # kernel 
        w = tf.Variable(tf.random.normal([in_ch, out_ch], 
                                              stddev=np.sqrt(2/in_ch)),
                             name='weight')
        # 추후 weights 만 꺼내 사용하기 위해 weights 을 따로 보관함.          
        tf.add_to_collection('weights', w)
        
        # bias 
        b = tf.Variable(tf.zeros([out_ch]), name='bias')

        # linear transformation 
        layer = tf.matmul(layer, w) + b

        # 마지막 층 , 즉 logit 이 아닐때 적용되는 것들.
        if not layer_idx == n_layers-1:
            # Dropout
            layer = tf.nn.dropout(layer, rate=drop_rate)

            # Activation
            layer = tf.nn.relu(layer)
            
            in_ch = out_ch
        
logits = layer

## LOSS

In [0]:
# weights 만 가져옵니다. 
weights = tf.get_collection('weights')

# CEE loss
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels)

# L1 Regularization : 0.5 * lambda *  ∑(weight)
l1_loss = tf.add_n([tf.reduce_sum(tf.abs(weight)) for weight in weights])*l1_lambda*0.5

# L2 Regularization : 0.5 * lambda * ∑(weight**2)
l2_loss = tf.add_n([tf.reduce_sum(weight**2) for weight in weights])*l2_lambda*0.5

# CEE + L1 or L2 Regularization
total_loss = tf.reduce_mean(loss) + l1_loss + l2_loss 

## Train op

In [0]:
## high level api을 이용해 Gradient Descent 구현
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

## next batch function

In [0]:
# 복원 추출 합니다. 
import random
def next_batch(xs, ys, batch_size):
    indices = random.sample(range(len(ys)), batch_size)
    return xs[indices], ys[indices]

## Metric

In [0]:
logits_cls = tf.argmax(logits, axis=1, output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(logits_cls, labels_cls), 
                     dtype=tf.float32))

## Hyperparam 설정


In [0]:
# batch size 
batch_size = 120 

#l2 loss decay
l2_decay = 0.0

#l1 loss decay
l1_decay = 0.0

#drop rate 
# 만약 0.1 이라면 0.1 의 확률로 unit 의 값을 0으로 만듭니다. 
drop_prob = 0.5

## Session Open

In [0]:
sess = tf.Session()

## Training


In [0]:
sess.run(tf.global_variables_initializer())
max_iter = 40000 
ckpt = 1000
mean_gradients=[]
accum_train_acc = [] 
accum_train_loss = [] 
accum_test_acc = [] 
accum_test_loss = [] 

for i in range(max_iter):
    # random select input data, labels
    batch_xs, batch_ys = next_batch(train_images, train_labels, batch_size)

    # Training 
    train_acc, train_loss, _ = sess.run([acc, total_loss, train_op], 
                                        feed_dict={x: batch_xs,
                                                   labels_cls: batch_ys,
                                                   learning_rate:0.1,                                                   
                                                   l2_lambda:l2_decay,
                                                   l1_lambda:l1_decay,
                                                   drop_rate:drop_prob,
                                                   phase_train:True
                                                  })
    # Evaluating
    if i % ckpt == 0:
        test_acc, test_loss = sess.run([acc, total_loss], 
                                    feed_dict={x:test_images,
                                               labels_cls: test_labels,
                                               drop_rate:0.0,
                                               phase_train: False
                                              })
        print('step : {} train_acc : {:.4f} train_loss : {:.4f} validation acc : {:.4f} validation loss : {:.4f}'.\
              format(i, train_acc, train_loss, test_acc, test_loss))
        
        # list 에 추출함.
        accum_train_acc.append(train_acc)
        accum_train_loss.append(train_loss)
        accum_test_acc.append(test_acc)        
        accum_test_loss.append(test_loss)        

## Visualization

In [0]:
plt.plot(range(0, max_iter, ckpt), accum_train_acc, label='train')
plt.plot(range(0, max_iter, ckpt), accum_test_acc, label='test')
plt.title('Accuracy')
plt.legend()


In [0]:
plt.plot(range(0,max_iter, ckpt), accum_train_loss, label='train')
plt.plot(range(0,max_iter, ckpt), accum_test_loss, label='test')
plt.title('Loss')
plt.legend()